# Performance Data Lookup

In [1]:
# System dependencies
from os import listdir
import time as clock
from IPython.display import clear_output

In [2]:
import pickle
import gzip
import re

In [3]:
# Data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
# Directory to save to
save_dir = '../data/continued/'

# Directory of prev job scans
id_dir = '../data/labels/IDs/'

In [5]:
# Directory of recent saved comet jobs
source_dir = '/oasis/projects/nsf/sys200/stats/xsede_stats/'

In [6]:
contents = listdir(source_dir)
contents

['archive_of_archive',
 'gordon_hostfile_logs',
 'gordon_pickles',
 'comet_accounting',
 'gordon_accounting',
 'comet_pickles',
 'archive',
 '.htaccess',
 'comet_hostfile_logs']

In [7]:
possible = [ source_dir+file_name for file_name in listdir(source_dir) ]

for item in possible:
    try:
        listdir(item)
    except:
        print(item)

/oasis/projects/nsf/sys200/stats/xsede_stats/gordon_hostfile_logs
/oasis/projects/nsf/sys200/stats/xsede_stats/gordon_pickles
/oasis/projects/nsf/sys200/stats/xsede_stats/gordon_accounting
/oasis/projects/nsf/sys200/stats/xsede_stats/comet_pickles
/oasis/projects/nsf/sys200/stats/xsede_stats/.htaccess
/oasis/projects/nsf/sys200/stats/xsede_stats/comet_hostfile_logs


In [8]:
locs = { 'aofa': source_dir+'archive_of_archive',
         'job_info': source_dir+'comet_accounting',
         'arc': source_dir+'archive'
         #'host_info': source_dir+'comet_hostfile_logs',
         #'old_pickles': source_dir+'comet_pickles'
       }

In [9]:
for key,loc in locs.items():
    print(loc)

/oasis/projects/nsf/sys200/stats/xsede_stats/archive_of_archive
/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting
/oasis/projects/nsf/sys200/stats/xsede_stats/archive


## Target Job

In [10]:
chosen = ('34583358', 'comet-15-11', '2020-07-08T18:25:03', '2020-07-10T18:25:19')

## Prep Cleaning

In [11]:
def get_time( spec=None ):
    if type(spec) is str:
        spec = float( spec )
    
    return clock.strftime("%Y-%m-%dT%H:%M:%S", clock.localtime( spec ))

In [12]:
def get_stamp( spec ):
    sf = "%Y-%m-%dT%H:%M:%S"
    return int(clock.mktime( clock.strptime( spec, sf ) ))

In [13]:
def check_static( alist ):
    return alist[1:] == alist[:-1]

In [14]:
def check_header( line ):
    if line.find(" ") < 0:
        try:
            return line[0] == '%'
        except:
            return False
        
    else:
        chunks = line.split(" ")
        try:
            return (chunks[0][0] == '%') or ( chunks[2].find("comet") >= 0 )
        except:
            return False

In [15]:
def check_job( chunk ):
    return chunk.find("-") == -1

In [16]:
def open_txt( txt_file ):
    
    with open( txt_file, "rt" ) as f:
        lines = f.readlines()
        f.close()
    
    return lines

In [17]:
def unzip_txt( gzipped ):
    
    with gzip.open( gzipped, 'rt') as f:
        lines = f.readlines()
        f.close()
    
    return lines

In [18]:
def quick_save( obj, label=get_time() ):
    
    try:
        out_file = open( label, 'wb')
        pickle.dump( obj, out_file)
        
        # double check save
        check_cpicore_set = pickle.load(open(cpiset_out, 'rb'))
        check_cpicore_set = None
        
    except:
        "There was a problem pickling the object - Save manually."

Data Munging

In [19]:
def info_dict( rules, info ):
    rules_list = rules.split("|")
    
    if len(rules_list) != len(info):
        return {}
    
    else:
        return { rules_list[i]:info[i] for i in range(len(rules_list)) }

In [20]:
def host_to_info_dict( zip_txt ):
    contents = unzip_txt( zip_txt )
    host_name = contents[1].partition(" ")[2][:11]
    out_dict = { host_name: {} }
    host_info = {}
    info_dict = { "Data":{},
                    "Job":"N/A",
                    "Schemas":{},
                    "Specs":[]
                }
    
    for line in contents:
            
        if line[0] == "$":
            info_dict["Specs"].append( format_spec( line ) )
            
        elif line[0] == "!":
            info_dict["Schemas"].update( format_schema( line ) )
        
        else:
            
            if (len(line) > 0) and (len(line) < 3 or check_header( line )):
                header_dict = format_header( line )
                
                if header_dict:
                    t = header_dict["Timestamp"]
                    host_info[ t ] = {}
                    
                    if check_job( header_dict["Jobid"] ):
                        info_dict["Job"] = { "Jobid": header_dict["Jobid"] } 
                    
            else:
                incoming = format_data( line )
                info_dict["Data"].update(incoming)
                
                host_info[t].update( info_dict )
                
    out_dict[host_name].update( host_info )
    
    return out_dict

In [21]:
def job_to_info_dict( txt_file_list ):
    nodes_by_date = {}
    unsaved = []

    for date in txt_file_list:
        try:
            # skip alt files
            #check_stamp = int( date[-14] )
            
            # read in file contents
            contents = open_txt( date )
            
            # formatting
            label = date[-14:-4]
            rules = contents[0]
            jobs = contents[1:]
            
            # template to save
            nodes_by_date[ label ] = {}
            nodes_by_date[ label ]["multiple"] = {}
            nodes_by_date[ label ]["rules"] = rules
            
            # run through lines in file
            for job in jobs:
                line = job.split("|")
                node = line[-1]
                info = info_dict( rules, line )
                
                # save multiple node jobs to specified loc
                if len(node) > 12:
                    nodes = format_nodelist( info )
                    for node in nodes:
                        nodes_by_date[ label ][ "multiple" ][ node ] = info
                
                else:
                    nodes_by_date[ label ][ node[:11] ] = info
        except:
            unsaved.append(date)
            
    
    return nodes_by_date, unsaved

Formatting

In [22]:
def format_header( line ):
    chunks = line.split(" ")
    
    try:
        if chunks[0][0] == '%':
            return {}
        else:
            return { "Timestamp": get_time( chunks[0] ), 
                     "Jobid": chunks[1],
                     "Host": chunks[2][:11] }
        
    except:
        return {}

In [23]:
def format_nodelist( nodelist ):
    purged = nodelist.replace('[','').replace(']','').replace(',','-').replace('-','').split("comet")[1:]
    nodes = []
    
    for item in purged:
        base = item[:2]
        prev = 2
        
        for i in range( 4,len(item)+1,2 ):
            node = 'comet' + '-' + base + '-' + item[ prev:i ]
            nodes.append(node)
            prev = i
    return nodes

In [24]:
def format_spec( line ):
    return line[1:-1]

In [25]:
def format_data( line ):
    chunks = line.split(" ")
    
    stat = chunks[0]
    dev = chunks[1]
    data = chunks[2:-1]
    
    return { (stat,dev): data }

In [26]:
def format_schema( line ):
    chunks = line.partition(" ")
    stat = chunks[0][1:]
    
    temp_sch = chunks[2:][0][:-1].replace(",E","").replace(",C","").split(" ")
    fin_sch = []
    
    for item in temp_sch:
        
        if item.find("=") > -1:
            new = item.replace(",","(") + ")"
            fin_sch.append( new )
        
        else:
            fin_sch.append( item )
    
    return { stat:fin_sch }

Data analysis

In [27]:
def timely_dict( host_data, host_name ):
    stamps = list(host_data[ host_name ].keys())
    schemas = host_data[ host_name ][ stamps[0] ]["Schemas"]
    timely_data = []
    
    for stamp in stamps:
        for key,data in host_data[ host_name ][ stamp ]["Data"].items():
            
            stat = key[0]
            dev = key[1]
            
            for i in range(len(data)):
                metric = schemas[stat][i]
            
            info = (stat, metric, dev, int(data[i]), stamp)
            timely_data.append( info )
    
    return timely_data

# Accessing Info

In [28]:
arc_data = [ locs['arc']+'/'+host_dir+'/'+stamp 
            for host_dir in listdir(locs['arc'])
            for stamp in listdir(locs['arc']+'/'+host_dir)  ]

aofa_data = [ locs['aofa']+'/'+host_dir+'/'+stamp 
            for host_dir in listdir(locs['aofa'])
            for stamp in listdir(locs['aofa']+'/'+host_dir)  ]

In [29]:
acct_info_locs = [ locs['job_info']+'/'+stamp for stamp in listdir(locs['job_info']) ]
dates = [ loc[-14:-4] for loc in acct_info_locs ]

In [30]:
### Process all ###
#arc_hosts = [ host_to_info_dict( host_file ) for host_file in arc_data[:1] ]
#acct_hosts = job_to_info_dict( acct_info_locs[:1] )

### Formatting Info

In [31]:
temp_host = host_to_info_dict( arc_data[0] )
temp_stamps = list(temp_host['comet-10-14'].keys())
focus = temp_host['comet-10-14'][temp_stamps[0]]

In [32]:
temp = acct_info_locs[0]
temp_acct = open_txt( temp )

In [33]:
focus['Job']

'N/A'

In [34]:
focus['Specs']

['tacc_stats 2.3.4',
 'hostname comet-10-14.sdsc.edu',
 'uname Linux x86_64 3.10.0-957.12.2.el7.x86_64 #1 SMP Tue May 14 21:24:32 UTC 2019',
 'uptime 4082659']

In [35]:
focus['Schemas']

{'block': ['rd_ios',
  'rd_merges',
  'rd_sectors(U=512B)',
  'rd_ticks(U=ms)',
  'wr_ios',
  'wr_merges',
  'wr_sectors(U=512B)',
  'wr_ticks(U=ms)',
  'in_flight',
  'io_ticks(U=ms)',
  'time_in_queue(U=ms)'],
 'cpu': ['user(U=cs)',
  'nice(U=cs)',
  'system(U=cs)',
  'idle(U=cs)',
  'iowait(U=cs)',
  'irq(U=cs)',
  'softirq(U=cs)'],
 'ib': ['excessive_buffer_overrun_errors(W=32)',
  'link_downed(W=32)',
  'link_error_recovery(W=32)',
  'local_link_integrity_errors(W=32)',
  'port_rcv_constraint_errors(W=32)',
  'port_rcv_data(U=4B(W=32)',
  'port_rcv_errors(W=32)',
  'port_rcv_packets(W=32)',
  'port_rcv_remote_physical_errors(W=32)',
  'port_rcv_switch_relay_errors(W=32)',
  'port_xmit_constraint_errors(W=32)',
  'port_xmit_data(U=4B(W=32)',
  'port_xmit_discards(W=32)',
  'port_xmit_packets(W=32)',
  'port_xmit_wait(U=ms(W=32)',
  'symbol_error(W=32)',
  'VL15_dropped(W=32)'],
 'ib_ext': ['port_select',
  'counter_select',
  'port_xmit_data(U=4B)',
  'port_rcv_data(U=4B)',
  'port

In [ ]:
org_data = {}

for host in temp_host.keys():
    for ts in temp_host[host]:
        for stat_key,data in temp_host[host][ts]['Data'].items():
            if stat_key not in org_data:
                org_data[stat_key] = [(ts,data)]
            else:
                org_data[stat_key].append( (ts,data) )
org_data

In [ ]:
test_timely = timely_dict( temp_host,'comet-10-14' )
test_timely

# Account Info 

In [ ]:
temp

In [ ]:
temp_acct

### Formatting Account Info

In [ ]:
rules = temp_acct[0].split("|")
rules_ex = temp_acct[1].split("|")
sample_acct = { rules[i]:rules_ex[i] for i in range(len(rules)) }
sample_acct